Pre-Activation Wide-ResNet Network for Spectrum Mining and Modulation Detection.

References:
    Basic ResNet: https://arxiv.org/pdf/1512.03385.pdf
    Pre-Act. ResNet: https://arxiv.org/pdf/1603.05027.pdf
    Wide-ResNet: https://arxiv.org/pdf/1605.07146v1.pdf

Below we create a decorator that wraps around our inference, loss, and optimizer subgraphs.
This makes sure we only call the graph creation code once. Small, but handy. Can extend this to work with `tf.variable_scope`. This combined with `partial` can really cut down on boilerplate code and make the network much more readable/understandable.

In [ ]:
import functools

def lazy_init(function):
    '''
    Lazy initialization for inference, loss, and optimzier in a graph.
    '''
    attribute = '_cache_' + function.__name__

    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            setattr(self, attribute, function(self))
        return getattr(self, attribute)

    return decorator

Next we create a named tuple for our hyperparameters. This is cleaner and less error-prone than passing in a dictionary. Combined with `FLAGS`, it is the ideal way of handling the multitude of parameters in a Deep Learning project.

We also define some constants for Batch Normalization that have shown to be reliable.

In [ ]:
from collections import namedtuple

# tuple for model hyperparameters
HyperParams = namedtuple('HyperParams',
                     'batch_size, num_cls, num_chans_in, height, width, filter_dims,'
                     'lr, kernel_dims, strides, n_hidden, relu_alpha')
# sample hyperparameters
h = HyperParams(128, 10, 1, 20, 512, [16, 16, 16, 32, 64], 1e-5, [3, 3],
                [1, 1, 1, 2, 2], 128, 0.01)

# parameters for batch_norm
_BATCH_NORM_DECAY = 0.997
_BATCH_NORM_EPSILON = 1e-5

Now we are ready to build on Pre-Activation ResNet in tensorflow:

In [5]:
import tensorflow as tf

def ResNet(object):
    def __init__(self, hps=h):
        '''
        Builds a 5-layer pre-activation ResNet.
        Using Xavier initializer for conv and dense weights.
        Leaky-relu as activation function.
        '''
        # parameters and inputs
        self.batch_size = 128 # hps.batch_size
        self.num_cls = 10 # hps.num_cls
        num_chans = 1 # hps.num_chans_in
        height = 20 # hps.height
        width = 512 # hps.width

        # placeholder for inputs, labels, and is_training flag
        self.inputs = tf.placeholder(tf.float32,
            shape=(self.batch_size, num_chans, height, width),
            name='x_placeholder')
        self.labels = tf.placeholder(tf.float32,
            shape=(self.batch_size, self.num_cls), name='labels')
        self.is_training = tf.placeholder_with_default(True, [],
            name='is_training')

        # network parameters
        self.n_filts = [16, 16, 16, 32, 64] # hps.filter_dims
        # self.n_filts = [16, 16, 80, 160, 320] # params for wide-resnet(5,4)
        self.kernel_dim = (3, 3) # hps.kernel_dims
        self.strides = [1, 1, 1, 2, 2] # hps.strides
        self.n_hidden = 128 # hps.n_hidden
        self.relu_alpha = 0.01 # hps.relu_alpha
        # learning rate
        self.lr = 1e-5 # hps.lr
        # initializers for weight layers
        self.conv_init = tf.contrib.layers.xavier_initializer_conv2d
        self.dense_init = tf.contrib.layers.xavier_initializer

        # forward pass through resnet
        self.inference = self._get_inference()
        # get loss from inference and labels
        self.loss = self._get_loss()
        # get optimizer step to minimize loss
        self.optim = self._get_optimizer()

    @lazy_init
    def _get_inference(self):
        x = self._get_first_layer()
        x = self._build_model(x)
        return x

    def _get_first_layer(self):
        '''
        Return the first convolutional layer common to all ResNet models.

        Note: Slightly different from layer + pooling in original resnet, but
            this performs better.
        '''
        with tf.variable_scope('first_layer'):
            x = tf.layers.conv2d(self.inputs, filters=self.n_filts[0],
                kernel_size=self.kernel_dim, strides=self.strides[0],
                kernel_initializer=tf.conv_init, padding='same')
            return x

    def _build_model(self, x):
        '''Builds the residual blocks.'''
        # scales better for deeper resnet blocks
        # for i in range(1, len(self.n_filts)):
        #     with tf.variable_scope('resnet_block_{:d}'.format(i)):
        #         x = self._resnet_block(x, self.n_filts[i], self.strides[i],
        #             self.training_pl)

        with tf.variable_scope('first_block'):
            x = self._resnet_block(x, self.n_filts[1], self.strides[1],
                    self.training_pl)

        with tf.variable_scope('second_block'):
            x = self._resnet_block(x, self.n_filts[2], self.strides[2],
                    self.training_pl)

        with tf.variable_scope('third_block'):
            x = self._resnet_block(x, self.n_filts[3], self.strides[3],
                    self.training_pl)

        with tf.variable_scope('fourth_block'):
            x = self._resnet_block(x, self.n_filts[4], self.strides[4],
                    self.training_pl)

        with tf.variable_scope('GAP_block'):
            # NOTE: assumes channels_first
            x = tf.reduce_mean(x, [2, 3])

        with tf.variable_scope('dense'):
            x = tf.reshape(x, [self.batch_size, -1])
            x = tf.layers.dense(x, self.n_hidden, activation=self._leaky_relu,
                    kernel_initializer=self.dense_init)

        with tf.variable_scope('logits'):
            x = tf.layers.dense(x, self.num_cls,
                    kernel_initializer=self.dense_init)

        return x

    @lazy_init
    def _get_loss(self):
        with tf.variable_scope('loss'):
            loss = tf.nn.softmax_cross_entropy_with_logits(
                labels=self.labels,
                logits=self.inference
            )
            loss = tf.reduce_mean(loss)
        return loss

    @lazy_init
    def _get_optimizer(self):
        with tf.variable_scope('optimizer'):
            optim = tf.train.MomentumOptimizer(learning_rate=self.lr,
                momentum=.9, use_nesterov=True)
            train_op = optim.minimize(self.loss)
        return train_op


    def _resnet_block(self, inputs, in_filts, strides, is_training):
        '''
        ResNet block with two convolutions.
        Projects original inputs through 1x1 conv if strides downsample.
        '''
        # store original x for skip-connection
        orig_in = inputs

        # if we are down-sampling, need to double amount of filters
        if strides == 2:
            in_filts *= 2

        # first pre-activation block
        inputs = self._batch_norm_relu(inputs, is_training)
        inputs = tf.layers.conv2d(inputs, in_filts, self.kernel_dim, strides,
            padding='same', data_format='channels_first',
            kernel_initializer=self.conv_init)

        # second pre-activation block
        inputs = self._batch_norm_relu(inputs, is_training)
        inputs = tf.layers.conv2d(inputs, in_filts, self.kernel_dim, strides,
            padding='same', data_format='channels_first',
            kernel_initializer=self.conv_init)

        # add back original inputs, with projection if needed
        if strides == 2:
            # 1x1 conv with downsample strides and filters
            orig_in = tf.conv2d(orig_in, in_filts, (1, 1), strides,
                padding='same', data_format='channels_first',
                kernel_initializer=self.conv_init)
        inputs += orig_in

        return inputs

    def _batch_norm_relu(self, inputs, is_training):
        '''
        Passes inputs through batch normalization and leaky-relu.
        '''
        inputs = tf.layers.batch_normalization(inputs, axis=3, momentum=_BATCH_NORM_DECAY,
            epsilon=_BATCH_NORM_EPSILON, center=True, scale=True, training=is_training,
            fused=True)
        inputs = self._leaky_relu(inputs)
        return inputs

    def _leaky_relu(self, inputs, alpha=0.01):
        return tf.maximum(self.relu_alpha * inputs, inputs)

In [3]:
tf.layers.conv2d_transpose??

In [ ]:
tf.losses.softmax_cross_entropy